In [1]:
import pandas as pd
import numpy as np
import pygtrie as trie
from collections import defaultdict
from scipy.stats import entropy

In [2]:
df = pd.read_csv("output_suffix.txt", sep=' ', header=None, names=["word","cnt","err"])

In [3]:
m = {} # 存储每个候选词的词频
m2 = defaultdict(lambda: defaultdict(int)) # 存储每个候选词的前缀字集合
for index, row in df.iterrows():
    m[row['word']] = row['cnt']
    if len(row['word']) == 1: continue # 单字词没有前缀字，无需计算
    key = row['word'][1:]
    prefix = row['word'][:1]
    m2[key][prefix] += row['cnt']

### 计算聚合度，聚合度定义为当前串频次与其后缀串频次的比值

In [4]:
def poly(x):
    if len(x['word']) == 1 or x['word'][1:] not in m: return 1 # 单字聚合度为1
    return x['cnt']/m[x['word'][1:]]

### 计算灵活度，灵活度定义为前缀字集合的信息熵

In [5]:
def flex(x):
    if x['word'] not in m2: return 1
    s = pd.Series(m2[x['word']])
    return entropy(s / s.sum())

In [6]:
df['poly'] = df.apply(poly, axis=1)
df['flex'] = df.apply(flex, axis=1)

In [7]:
df['score'] = df.apply(lambda x: x['flex']*x['poly'], axis=1)

In [8]:
d1 = df[(df['score']>1) & (df['err']==0)].sort_values(['score'], ascending=False)

In [9]:
d1

,word,cnt,err,poly,flex,score
13,房地产开发有限公司,4346,0,0.992691,4.614287,4.580560
3,有限公司,84246,0,0.999941,3.932106,3.931873
17,科技有限公司,3990,0,0.999749,3.692177,3.691252
7,有限责任公司,5814,0,1.000000,3.572008,3.572008
20,厂,3143,0,1.000000,3.476868,3.476868
22,贸易有限公司,2889,0,0.998272,3.474757,3.468753
66,建设工程有限公司,765,0,1.000000,3.313624,3.313624
61,建筑工程有限公司,895,0,1.000000,3.273912,3.273912
25,集团有限公司,2622,0,0.999238,3.211667,3.209219
89,建设集团有限公司,583,0,1.000000,3.195880,3.195880


In [13]:
t = trie.CharTrie()
for _, row in d1.iterrows():
    t[row['word'][::-1]] = row

In [14]:
for _, row in d1.iterrows():
    rword = row['word'][::-1]
    ret = t.prefixes(rword)
    parent = None
    maxlen = -1
    for k, v in ret:
        if k == rword: continue
        if len(k) > maxlen:
            maxlen = len(k)
            parent = k[::-1]
    if parent is not None:
        print(parent, row['word'])

有限公司 房地产开发有限公司
有限公司 科技有限公司
有限公司 贸易有限公司
工程有限公司 建设工程有限公司
工程有限公司 建筑工程有限公司
有限公司 集团有限公司
集团有限公司 建设集团有限公司
有限公司 股份有限公司
有限公司 机械有限公司
有限公司 建设有限公司
有限公司 服饰有限公司
有限公司 物流有限公司
有限公司 设备有限公司
有限公司 电子有限公司
有限公司 工程有限公司
制造有限公司 机械制造有限公司
管理有限公司 投资管理有限公司
有限责任公司 房地产开发有限责任公司
有限公司 纺织有限公司
有限公司 房地产有限公司
股份有限公司 科技股份有限公司
有限公司 技术有限公司
有限公司 化工有限公司
有限公司 发展有限公司
贸易有限公司 汽车贸易有限公司
有限公司 制造有限公司
有限公司 商贸有限公司
有限公司 管理有限公司
发展有限公司 科技发展有限公司
服务有限公司 汽车销售服务有限公司
有限公司 钢铁有限公司
有限公司 建筑劳务有限公司
有限公司 投资有限公司
有限公司 材料有限公司
工程有限公司 装饰工程有限公司
有限公司 咨询有限公司
工程有限公司 建筑安装工程有限公司
运输有限公司 汽车运输有限公司
有限公司 玻璃有限公司
股份有限公司 融资担保股份有限公司
有限公司 制衣有限公司
有限公司 运输有限公司
有限公司 混凝土有限公司
有限公司 (集团)有限公司
设备有限公司 机械设备有限公司
有限公司 房地产经纪有限公司
贸易有限公司 国际贸易有限公司
有限公司 轴承有限公司
发展有限公司 投资发展有限公司
有限公司 服务有限公司
担保有限公司 融资担保有限公司
管理有限公司 酒店管理有限公司
建材有限公司 新型建材有限公司
有限公司 建材有限公司
担保有限公司 投资担保有限公司
制造有限公司 设备制造有限公司
有限公司 进出口有限公司
制品有限公司 金属制品有限公司
科技有限公司 生物科技有限公司
厂 加工厂
科技有限公司 电子科技有限公司
有限责任公司 建设有限责任公司
有限公司 投资开发有限公司
有限公司 (北京)有限公司
有限公司 租赁有限公司
有限公司 工贸有限公司
有限公司 顾问有限公司
有限责任公司 贸易有限责任公司
有限公司 销售有限公司
有限公司 娱乐有限公司
有限责任公司 工程有限责任公司
有限公司 汽车配件有限公司
厂 